In [14]:
!date
!python --version

Thu Apr 17 07:55:26 AM UTC 2025
Python 3.11.12


# トピックモデルによるクラスタリング
トピックモデルとは文書中の単語出現分布を元に傾向（≒トピックらしきもの）を観察しようとするアプローチで、クラスタリングの一種である。なお、一般的なクラスタリング（例えば[k平均法](https://ja.wikipedia.org/wiki/K平均法)）では一つのサンプルが一つのクラスタに属するという前提でグルーピングを行うのに対し、トピックモデルでは一つのサンプルが複数のクラスタを内包しているという前提でグルーピングを行う。次の例を眺めるとイメージをつかみやすいだろう。

- 例1: [トピックモデル入門：WikipediaをLDAモデル化してみた](https://recruit.gmo.jp/engineer/jisedai/blog/topic-model/)
- 例2: [Wikipedia: Topic model](https://en.wikipedia.org/wiki/Topic_model)

基本的には文書を BoW (CountVectrizor) やそれの重みを調整した TF-IDF 等の「文書単語行列」を作成し、ここから文書館類似度や単語間類似度を元に集約（≒次元削減）を試みる。文書単語行列の作成方法や次元削減方法、類似度の求め方などで様々なアルゴリズムが提案されている。ここでは (1) BowベースのLDAと、(2) TF-IDFベースのLDAを行い、それぞれどのようなトピックが出てくるのか眺めてみよう。

なお、トピックモデルの注意点として、**トピックそのものは人手による解釈が求められる** 点が挙げられる。例えば先に上げた[トピックモデル入門：WikipediaをLDAモデル化してみた](https://recruit.gmo.jp/engineer/jisedai/blog/topic-model/)における図2（下図）では「政治」「スポーツ」「国際」といったトピックが並んでいるが、実際には「4-1. トピック観察」を行う必要がある。実際に観察してみよう。

In [15]:
# spacy, ginza インストール
!pip install -U ginza ja_ginza

# plotlyで作図した図をファイル出力するためのパッケージ
#!pip install -U kaleido

## データの準備
これまで見てきたいつものやつ。

In [16]:
!curl -O https://ie.u-ryukyu.ac.jp/~tnal/2022/dm/static/r_assesment.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34834  100 34834    0     0  11287      0  0:00:03  0:00:03 --:--:-- 11287


In [17]:
import collections

import numpy as np
import pandas as pd
import spacy
from wordcloud import WordCloud

nlp = spacy.load("ja_ginza")

assesment_df = pd.read_pickle('r_assesment.pkl')
assesment_df.head()

,title,grade,required,q_id,comment
0,工業数学Ⅰ,1,True,Q21 (1),特になし
1,工業数学Ⅰ,1,True,Q21 (2),正直わかりずらい。むだに間があるし。
2,工業数学Ⅰ,1,True,Q21 (2),例題を取り入れて理解しやすくしてほしい。
3,工業数学Ⅰ,1,True,Q21 (2),特になし
4,工業数学Ⅰ,1,True,Q21 (2),スライドに書く文字をもう少しわかりやすくして欲しいです。


In [18]:
# 分かち書き
poses = ['PROPN', 'NOUN', 'VERB', 'ADJ', 'ADV'] #名詞、動詞、形容詞、形容動詞

assesment_df['wakati'] = ''
for index, comment in enumerate(assesment_df['comment']):
    doc = nlp(comment)
    wakati_words = []
    for token in doc:
        if token.pos_ in poses:
            wakati_words.append(token.lemma_)
    wakati_text = ' '.join(wakati_words)
    assesment_df.at[index, 'wakati'] = wakati_text

assesment_df

,title,grade,required,q_id,comment,wakati
0,工業数学Ⅰ,1,True,Q21 (1),特になし,特に なし
1,工業数学Ⅰ,1,True,Q21 (2),正直わかりずらい。むだに間があるし。,正直 わかる ずらい むだ 間 ある
2,工業数学Ⅰ,1,True,Q21 (2),例題を取り入れて理解しやすくしてほしい。,例題 取り入れる 理解 する
3,工業数学Ⅰ,1,True,Q21 (2),特になし,特に なし
4,工業数学Ⅰ,1,True,Q21 (2),スライドに書く文字をもう少しわかりやすくして欲しいです。,スライド 書く 文字 もう 少し わかる する
...,...,...,...,...,...,...
165,データマイニング,3,False,Q22,課題が難しいものが多く、時間を多くとってもらえたのは非常に良かったですがかなりきつかったです...,課題 難しい もの 多い 時間 多い とる もらえる 非常 良い かなり きつい ござる
166,ICT実践英語Ⅰ,3,False,Q22,オンラインなどで顔を合わせてやりたかったです。,オンライン 顔 合わせる やる
167,知能情報実験Ⅲ,3,True,Q21 (2),unityの操作方法の説明などを最初に行ってもらえたらもう少しスムーズにできたのではないかと思う。,unity 操作方法 説明 最初 行く もらえる もう 少し スムーズ できる 思う
168,知能情報実験Ⅲ,3,True,Q22,それぞれに任せるといった形で進められたものだったのでそれなりに進めやすかったですが、オンライ...,それぞれ 任せる いう 形 進める もの なり 進める オンライン 班 員 指導 全く する...


## 文書ベクトルの作成
ここでは CountVectorizer (Bag-of-Words) で作成してみよう。

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

stop_words = ['こと', '\r\n', 'ため', '思う', 'いる', 'ある', 'する', 'なる']
vectorizer = CountVectorizer(stop_words=stop_words)
bow_tf_vector = vectorizer.fit_transform(assesment_df['wakati'])
print('bow_tf_vector.shape = ', bow_tf_vector.shape)

bow_tf_vector.shape =  (170, 741)


## LDAによるトピックモデル解析
sklearnでは [LatentDirichletAllocation](https://scikit-learn.org/stable/modules/decomposition.html?highlight=lda#latent-dirichlet-allocation-lda) として用意されている。

In [20]:
from sklearn.decomposition import LatentDirichletAllocation

NUM_TOPICS = 5 #トピック数
max_iter = 100  #LDAによる学習回数
lda = LatentDirichletAllocation(n_components=NUM_TOPICS,
                                max_iter=max_iter,
                                learning_method='online',
                                random_state=123) # シード値を指定すると結果を再現できる
data_lda = lda.fit_transform(bow_tf_vector)

## トピックの観察


In [21]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_top_words(model, feature_names, n_top_words, title):
    """
    LDA のトピックごと上位語を水平方向のバーで表示する Plotly 版

    Parameters
    ----------
    model : sklearn.decomposition.LatentDirichletAllocation
        すでに fit_transform 済みの LDA モデル
    feature_names : array‑like, shape (n_features,)
        model.get_feature_names_out() で得た語彙
    n_top_words : int
        各トピックで表示したい単語数
    title : str
        図全体のタイトル
    """
    n_topics = model.components_.shape[0]
    n_cols = 5                                # 列数は固定
    n_rows = int(np.ceil(n_topics / n_cols))  # トピック数に応じて行数を決定

    # サブプロット用の Figure を用意
    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        shared_xaxes=False,
        horizontal_spacing=0.08,
        vertical_spacing=0.06,
        subplot_titles=[f"Topic {i + 1}" for i in range(n_topics)],
    )

    for topic_idx, topic in enumerate(model.components_):
        # 指定トピックの上位語と重み
        top_idx = topic.argsort()[-n_top_words:]
        top_features = [feature_names[i] for i in top_idx]
        weights = topic[top_idx]

        row = topic_idx // n_cols + 1
        col = topic_idx % n_cols + 1

        # 水平バーを追加
        fig.add_trace(
            go.Bar(
                x=weights,
                y=top_features,
                orientation="h",
                marker=dict(line=dict(width=0)),  # 枠線を消してすっきり
            ),
            row=row,
            col=col,
        )

        # y 軸を上から下に並べ替え（matplotlib の barh と同じ見た目）
        fig.update_yaxes(autorange="reversed", row=row, col=col)

    # 図全体のレイアウト調整
    fig.update_layout(
        height=450 * n_rows,
        width=1700,
        title=dict(text=title, x=0.5, xanchor="center", font=dict(size=40)),
        showlegend=False,
        margin=dict(t=120, l=20, r=20, b=20),
    )

    # サブプロットタイトル（各トピック）のフォントサイズを揃える
    fig.update_annotations(font_size=22)

    fig.show()
    #file_title = title.replace(' ', '_')
    #fig.write_image(f'{file_title}.png')

In [22]:
n_top_words = 10
plot_top_words(lda, vectorizer.get_feature_names_out(), n_top_words, "Topics in LDA model (TF)")

## 文書ベクトル2(TF-IDF）

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

stop_words = ['こと', '\r\n', 'ため', '思う', 'いる', 'ある', 'する', 'なる']
vectorizer2 = TfidfVectorizer(stop_words=stop_words)
tfidf_vector = vectorizer2.fit_transform(assesment_df['wakati'])
print('tfidf_vector.shape = ', tfidf_vector.shape)

tfidf_vector.shape =  (170, 741)


In [24]:
lda2 = LatentDirichletAllocation(n_components=NUM_TOPICS,
                                max_iter=max_iter,
                                learning_method='online',
                                random_state=123) # シード値を指定すると結果を再現できる

data_lda2 = lda2.fit_transform(bow_tf_vector)

In [25]:
plot_top_words(lda2, vectorizer2.get_feature_names_out(), n_top_words, "Topics in LDA model (TF-IDF)")